# EDA of Sales of summer clothes in E-commerce Wish dataset

First of all we will build a function to analyse the sales data for us and provide important information such as:
* Data type of each field
* Which columns has missing data and number of missing records in each column
* What is the correlation of all the other numeric columns with the target column 
* Now for the numeric features: the mean, median and mode
* For categorical columns: mode


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
product_sales_df = pd.read_csv('/kaggle/input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv')
cat_count_df = pd.read_csv('/kaggle/input/summer-products-and-sales-in-ecommerce-wish/unique-categories.sorted-by-count.csv')

In [ ]:
product_sales_df.head()

In [ ]:
product_sales_df.columns

In [ ]:
target_col = 'units_sold'

print(f"Shape of dataframe {product_sales_df.shape}")

rows = []
for col in product_sales_df.columns:
    if product_sales_df[col].isin([0, 1, np.nan]).all():
        row_dict = {'ColumnName': col, 'DataType': 'binary', 'HasMissing':product_sales_df.isnull().any().loc[col],
                    'NumberOfMissingCells': product_sales_df.isnull().sum().loc[col], 'CorrelationWithTarget': product_sales_df.corr()[target_col].loc[col],
                    'Mean': np.nan, 'Median': np.nan,'Mode': product_sales_df.mode()[col].loc[0], 'MinValue': np.nan, 'MaxValue': np.nan }
    
    elif product_sales_df.dtypes.loc[col] == 'int64' or product_sales_df.dtypes.loc[col] == 'float64':
        
        row_dict = {'ColumnName': col, 'DataType': product_sales_df.dtypes.loc[col], 'HasMissing':product_sales_df.isnull().any().loc[col],
                    'NumberOfMissingCells': product_sales_df.isnull().sum().loc[col], 'CorrelationWithTarget': product_sales_df.corr()[target_col].loc[col],
                    'Mean': product_sales_df.mean().loc[col], 'Median':product_sales_df.median().loc[col], 'Mode': product_sales_df.mode()[col].loc[0],
                    'MinValue': product_sales_df.min().loc[col], 'MaxValue': product_sales_df.max().loc[col] }
        
    else:
        row_dict = {'ColumnName': col, 'DataType': product_sales_df.dtypes.loc[col], 'HasMissing':product_sales_df.isnull().any().loc[col],
                    'NumberOfMissingCells': product_sales_df.isnull().sum().loc[col], 'CorrelationWithTarget': np.nan, 'Mean': np.nan, 'Median': np.nan,
                    'Mode': product_sales_df.mode()[col].loc[0], 'MinValue': np.nan, 'MaxValue': np.nan }
        
    rows.append(row_dict)
    
        
info_df = pd.DataFrame(rows, columns=['ColumnName', 'DataType', 'HasMissing', 'NumberOfMissingCells', 'CorrelationWithTarget', 'Mean', 'Median', 'Mode', 'MinValue', 'MaxValue'])

info_df.set_index('ColumnName', inplace=True)
    
info_df = info_df.sort_values('CorrelationWithTarget', ascending=False, na_position='last')

print("FOR NUMERICAL COLUMNS")
info_df[info_df['DataType']!='object']

In [ ]:
print("\nFor categorical/non-numeric columns")
info_df[info_df['DataType']=='object'].drop(['CorrelationWithTarget', 'Mean', 'Median', 'MinValue', 'MaxValue'], axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(product_sales_df.corr()[[target_col]].sort_values(by=target_col, ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title(f'Features Correlating with {target_col}', fontdict={'fontsize':18}, pad=16);

## Let's first start with categorical features and process them

### 1. currency_buyer
It has no missing values and the only value is 'EUR'

In [ ]:
product_sales_df['currency_buyer'].unique()

## 2. product_color
Has 41 missing values and most common value is black

In [ ]:
count = product_sales_df['product_color'].value_counts()
count

In [ ]:
np.sort(product_sales_df['product_color'].dropna().unique())

There are so many colours, let's see if we can combine different shades of a colour into one colour like : example navy blue, blue and light blue into just blue

In [ ]:
product_sales_df['product_color'] = product_sales_df['product_color'].str.lower()

In [ ]:
product_sales_df[product_sales_df['product_color'].str.contains('&', na=False)]['product_color'].unique()

In [ ]:
shade_to_colour = {
    'navyblue': 'blue', 'lightblue': 'blue', 'skyblue': 'blue', 'lakeblue': 'blue', 'darkblue': 'blue', 'denimblue': 'blue', 'navy blue': 'blue', 'prussianblue': 'blue',
    'navy': 'blue',
    'armygreen': 'green', 'army green': 'green', 'fluorescentgreen': 'green', 'mintgreen': 'green', 'light green': 'green', 'lightgreen': 'green',
    'applegreen': 'green', 'darkgreen': 'green', 'army': 'green', 'khaki': 'green', 'lightkhaki': 'green',
    'lightyellow': 'yellow', 
    'winered': 'red', 'wine red': 'red', 'lightred': 'red', 'coralred': 'red', 'rose red': 'red', 'watermelonred': 'red', 'orange-red': 'red', 'rosered': 'red',
    'claret': 'red', 'burgundy': 'red', 
    'gray': 'grey', 'silver': 'grey','lightgray': 'grey', 'lightgrey': 'grey', 'greysnakeskinprint': 'grey',
    'coffee': 'brown', 'camel': 'brown', 'tan': 'brown', 
    'offwhite': 'white', 'ivory': 'white', 'nude': 'white',
    'lightpink': 'pink', 'dustypink':'pink', 'rosegold': 'pink',
    'lightpurple': 'purple', 'coolblack': 'black', 'apricot': 'orange', 'offblack': 'black'
}

def update_color(col):
    if shade_to_colour.get(col, False):
        return shade_to_colour.get(col)
    elif '&' in col:
        return 'dual'
    elif col in shade_to_colour.values():
        return col
    else:
        return 'other'

product_sales_df['product_color'].replace(np.nan, 'others', inplace=True)

product_sales_df['product_color'] = product_sales_df.product_color.apply(update_color)

In [ ]:
count = product_sales_df['product_color'].value_counts()
count

In [ ]:
col_df = product_sales_df.groupby('product_color').agg('sum')['units_sold'].to_frame()
col_df.reset_index(level=0, inplace=True)
col_df

In [ ]:
fig = plt.gcf()
fig.set_size_inches( 16, 10)
sns.barplot(x="product_color", y="units_sold", data=col_df)

After applying the necesssary transformation of the colour column we can see that black has sold most units followe by white.

## Tags
No missing values
 

In [ ]:
product_sales_df['tags_count'] = product_sales_df['tags'].str.split(',').str.len()

fig = plt.gcf()
fig.set_size_inches( 16, 10)
sns.lineplot(data=product_sales_df, x="tags_count", y="units_sold", ci=None)

Products with tags more than 35 are more discoverable and are thus bought more often.There is a sudden spike at just below 10 tags so let's investigate if that's an outlier


In [ ]:
product_sales_df[product_sales_df['tags_count']<=10]

As we can see there are only 27 products with tags count less than 10 and only 2 with sales of 20000 and rest have sales like 50, 100, 1000, 5000
So these two are outlires and thus reson for spike

Let's have a look at the most common tags with the help of a wordcloud

In [ ]:
from wordcloud import WordCloud
ustr = " ".join(product_sales_df['tags'].str.lower().str.split(',').sum())

fig = plt.gcf()
fig.set_size_inches( 16, 10)
wordcloud = WordCloud(background_color='white').generate(ustr) 
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

## 3. product_variation_size_id
14 missing values
most common value is 'S'

In [ ]:
product_sales_df['product_variation_size_id'].unique()

In [ ]:
product_sales_df['product_variation_size_id'].value_counts().head(50)

Lets try to reduce the number of sizes here. 

In [ ]:
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].str.lower().str.replace('.', '').str.replace('size--', '').str.replace('size -', '').str.replace('size/', '').str.replace('size ', '').str.replace('size-', '')

In [ ]:
product_sales_df['product_variation_size_id'].unique()

As we can still few are left.

In [ ]:
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('2xl', 'xl')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('3xl', 'xl')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('4xl', 'xl')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('5xl', 'xl')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('6xl', 'xl')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('x   l', 'xl')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('sizel', 'l')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('size4xl', 'xl')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('x   l', 'xl')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace('1 pc - xl', 'xl')

In [ ]:
def change_size(cl):
    if cl in 'xl,l,s,xs,m,xxl,xxxs,xxxxxl,xxxxl'.split(','):
        return cl
    else:
        return 'other'

product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].replace(np.nan, 'OTHER')
product_sales_df['product_variation_size_id'] = product_sales_df['product_variation_size_id'].apply(change_size)

In [ ]:
fig = plt.gcf()
fig.set_size_inches( 16, 10)
sns.barplot(x="product_variation_size_id", y="units_sold", data=product_sales_df)

In [ ]:
fig = plt.gcf()
fig.set_size_inches( 16, 10)
sns.countplot('product_variation_size_id',
              order = product_sales_df['product_variation_size_id'].value_counts().index,
              data = product_sales_df)
plt.show()

Here we can see that the majority of the products are of size 'S' but size 'M' has the most units_sold

shipping_option_name

In [ ]:
product_sales_df.groupby('shipping_option_name').agg(['count', 'sum'])['units_sold']

In [ ]:
fig = plt.gcf()
fig.set_size_inches( 25, 16)
sns.barplot(x="shipping_option_name", y="units_sold", data=product_sales_df)
plt.show()

## urgency_text

In [ ]:
product_sales_df.urgency_text.value_counts()

Let's drop  urgency_text,
title, title_orig, currency_buyer, urgency_tex, merchant_title,merchant_name, 
merchant_id,merchant_profile_picture,product_url	object, product_picture, 
product_id, theme and crawl_month

## origin_country

In [ ]:
product_sales_df.groupby('origin_country').agg(['count', 'sum'])['units_sold']

In [ ]:
list_of_na_merchants = product_sales_df[product_sales_df['origin_country'].isna()]['merchant_id'].values

for m in list_of_na_merchants:
    print("merchant title " + m)
    print(product_sales_df[product_sales_df['merchant_id']==m]['origin_country'])

So merchants which have missing origin country cannot be replaced by looking at the origin_country of the same merchant for another product.
This is based on the assumnption that a country of origin for a merchant should have the same value across product.

We will replace this with CN as its the most frequent value.


In [ ]:
product_sales_df['origin_country'].fillna('CN', inplace=True)


We are done with processing the categorical columns.
Now lets start  with the numerical data

The first column is 'units_sold' which is our target variable. But we don't want to predict the number of sales for a product but rather if the product has been successful on the Wish.com platform.
So we will start by converting the numerical data to a binary one. (successful or not)
units_sold has no missing values and has a median of 1000. So we will consider products to be successful if they have sales greater than 1000.
Now we are using median because because it not affected by very high outliers

## units_sold

In [ ]:
product_sales_df['success'] = product_sales_df['units_sold'].apply(lambda x: 1 if x>1000 else 0)

## rating_count
no missing values, highest correlation with target

In [ ]:
fig = plt.gcf()
fig.set_size_inches( 16, 10)
sns.scatterplot(data=product_sales_df, x="rating_count", y="units_sold", hue='success', size='units_sold', sizes=(10, 200))

Thus we can see that the number of rating affects the success of a product greatly.

## 'rating_five_count', 'rating_four_count', 'rating_three_count','rating_two_count', 'rating_one_count'

They all have very high correlation with the target variable and have 45 missing values each.
It turns out that these products with missing values for these 5 columns have no ratings (rating_count=0) 
So let's replace these values with zero. Also we find that the value of 'rating' column for these products is 5 which is unlikely given the products have zero rating


In [ ]:
product_sales_df.update(product_sales_df[['rating_five_count', 'rating_four_count', 'rating_three_count','rating_two_count', 'rating_one_count']].fillna(0))
product_sales_df.loc[product_sales_df['rating_count']==0, 'rating'] = 0

Our next step is to normalise these columns as rating count for 'five' has a similar correlation with rating count for 'one' 
This is unlikely as product with low ratings is less likely to be successful

In [ ]:
product_sales_df[['rating_five_count', 'rating_four_count', 'rating_three_count','rating_two_count', 'rating_one_count', 'rating_count', 'rating']].head()

In [ ]:
sns.barplot(x='success', y='rating_five_count', data=product_sales_df)

In [ ]:
sns.barplot(x='success', y='rating_one_count', data=product_sales_df)

In [ ]:
product_sales_df['rating_three_count_prop'] = product_sales_df['rating_three_count']/product_sales_df['rating_count']
product_sales_df['rating_four_count_prop'] = product_sales_df['rating_four_count']/product_sales_df['rating_count']
product_sales_df['rating_five_count_prop'] = product_sales_df['rating_five_count']/product_sales_df['rating_count']
product_sales_df['rating_two_count_prop'] = product_sales_df['rating_two_count']/product_sales_df['rating_count']
product_sales_df['rating_one_count_prop'] = product_sales_df['rating_one_count']/product_sales_df['rating_count']
# to remove nan due to zero division
product_sales_df.update(product_sales_df[['rating_five_count_prop', 'rating_four_count_prop', 'rating_three_count_prop','rating_two_count_prop', 'rating_one_count_prop']].fillna(0))


In [ ]:
sns.barplot(x='success', y='rating_one_count_prop', data=product_sales_df)

In [ ]:
sns.barplot(x='success', y='rating_five_count_prop', data=product_sales_df)

In [ ]:
sns.barplot(x='success', y='rating_four_count_prop', data=product_sales_df)

Thus normalising the data really solves the problem

## merchant_rating_count

high correlation and no missing values

## merchant_has_profile_picture


In [ ]:
fig = plt.gcf()
fig.set_size_inches( 16, 10)
sns.countplot(data=product_sales_df, x='merchant_has_profile_picture', hue='success')

We can see that a merchant is more likely to be successful if he has a profile picture
Thus the correlation with units_sold is high too

## merchant_rating

In [ ]:
fig = plt.gcf()
fig.set_size_inches( 16, 10)
sns.violinplot(data=product_sales_df, y='merchant_rating', x='success')

In [ ]:
product_sales_df['has_urgency_banner'] = product_sales_df['has_urgency_banner'].fillna(0)

In [ ]:
## dropping unnecessary columns
product_sales_df = product_sales_df.drop(['crawl_month','product_id','product_picture', 'product_url', 'merchant_profile_picture', 'merchant_id',
                       'currency_buyer', 'theme','urgency_text', 'merchant_title', 'merchant_name', 'merchant_info_subtitle',
                      'title','title_orig','tags', 'shipping_option_name', "inventory_total" , "badge_fast_shipping" ,
                       "badge_local_product" , "shipping_is_express", "units_sold"], axis = 1)



In [ ]:
product_sales_df = product_sales_df.drop(["rating_five_count","rating_four_count","rating_three_count", "rating_two_count", "rating_one_count"], axis = 1)

In [ ]:
product_sales_df = pd.get_dummies(product_sales_df, 
                           columns = ['product_color'],
                           prefix = 'color_',
                           drop_first = True)
product_sales_df.head()

In [ ]:
product_sales_df = pd.get_dummies(product_sales_df, 
                           columns = ['product_variation_size_id'],
                           prefix = 'size_',
                           drop_first = True)
product_sales_df.head()

In [ ]:
product_sales_df = pd.get_dummies(product_sales_df, 
                           columns = ['origin_country'],
                           prefix = 'country_',
                           drop_first = True)
product_sales_df.head()

In [ ]:
product_sales_df.dtypes

In [ ]:
product_sales_df.isna().any().to_frame()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(product_sales_df.drop(['success'], axis=1), product_sales_df['success'],
test_size=0.2, random_state=1, stratify=product_sales_df['success'])


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_pred, y_test)

Random Forests Classifier gives us a really good accuracy of 96% on the test set. But let's check this with k fold cross validation and see if there is overfitting.
We can also checkout the important features (columns) in this dataset using the RandomForestClassifier as decision trees are one of the great ways to do so.

In [ ]:
features = pd.DataFrame()
features['feature'] = X_train.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

features.plot(kind='barh', figsize=(25, 25))

It looks like rating count has too much effect on our target variable.
Lets try to remove this see the graph again

In [ ]:
features = pd.DataFrame()
features['feature'] = X_train.columns
features['importance'] = clf.feature_importances_
features = features[features['feature']!='rating_count']
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

features.plot(kind='barh', figsize=(25, 25))

Lets try removing the unnecessary features and see how it affects our accuracy. We will be using Sklearn's SelectFromModel

In [ ]:
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(clf, prefit=True)
train_reduced = model.transform(X_train)
print(train_reduced.shape)

test_reduced = model.transform(X_test)
print(test_reduced.shape)

The result are not that different, lets try PCA and see if we can improve the accuracy 

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA

# sc = StandardScaler()
# train_scaled = sc.fit_transform(X_train)
# test_scaled = sc.transform(X_test)

# pca = PCA(n_components=20)
# train_reduced = pca.fit_transform(train_scaled)
# test_reduced = pca.transform(test_scaled)
# print(train_reduced.shape)
# print(test_reduced.shape)

We will now fit our data into diferent classfiers and choose the best model

In [ ]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score

In [ ]:
def compute_score(clf, X, y, scoring='accuracy'):
    xval = cross_val_score(clf, X, y, cv = 5, scoring=scoring)
    return np.mean(xval)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

logreg = LogisticRegression()
logreg_cv = LogisticRegressionCV()
rf = RandomForestClassifier()
gboost = GradientBoostingClassifier()
gnb = GaussianNB()
svm = SVC()
knn = KNeighborsClassifier()
xgboost = XGBClassifier()
dc = DecisionTreeClassifier()
adc = AdaBoostClassifier()
models = [logreg, logreg_cv, rf, gboost, gnb, svm, knn, xgboost, dc, adc]

for model in models:
    print('Cross-validation of : {0}'.format(model.__class__))
    score = compute_score(clf=model, X=train_reduced, y=y_train, scoring='accuracy')
    print('CV score = {0}'.format(score))
    print('****')

In [ ]:
clf = XGBClassifier()
clf.fit(train_reduced, y_train)

y_pred = clf.predict(test_reduced)

accuracy_score(y_pred, y_test)